In [ ]:
import os
import cv2
from pathlib import Path
import open3d as o3d
import numpy as np
import rootutils
import matplotlib.pyplot as plt

from helpers.segmentation_inference import SegmentationInferenceOnnx

rootutils.setup_root(
    os.path.abspath(''), indicator=['.git', 'pyproject.toml'], pythonpath=True
)

from src.stereo_inference import StereoInferenceOnnx

from src.stereo_calibrate import (
    read_stereo_calibration,
    rectify_image,
    read_camera_intrinsics,
)
from src.utils import (
    images_to_tensors,
    visualize_disparity,
    postprocess_disparity,
    create_point_cloud,
)

In [ ]:
# Load stereo calibration parameters
stereo_map_left, stereo_map_right = read_stereo_calibration(
    '../camera_configs/stereo_calibration.xml'
)
# Read camera intrinsics
K, distance_between_cameras = read_camera_intrinsics(
    '../camera_configs/left_camera_intrinsics.xml'
)
# Load stereo model
stereo_model = StereoInferenceOnnx('../models/fs_800_640.onnx')

In [ ]:
segmentation_inference = SegmentationInferenceOnnx(
    model_path='../models/unet++_lab_seat_segmentation.onnx'
)

In [ ]:
# load images
image_name = '20250423_164007_15'
left_image = cv2.imread('../data/left/left_' + image_name + '.png', cv2.IMREAD_COLOR)
right_image = cv2.imread('../data/right/right_' + image_name + '.png', cv2.IMREAD_COLOR)

# Rectify images
left_rectified = rectify_image(left_image, stereo_map_left)
right_rectified = rectify_image(right_image, stereo_map_right)

# bgr to rgb and resize images to the model input size
left_rectified = cv2.cvtColor(left_rectified, cv2.COLOR_BGR2RGB)
right_rectified = cv2.cvtColor(right_rectified, cv2.COLOR_BGR2RGB)
left_rectified = cv2.resize(
    left_rectified,
    (stereo_model.input_width, stereo_model.input_height),
    interpolation=cv2.INTER_LINEAR,
)
right_rectified = cv2.resize(
    right_rectified,
    (stereo_model.input_width, stereo_model.input_height),
    interpolation=cv2.INTER_LINEAR,
)
scale = min(
    left_rectified.shape[0] / left_image.shape[0],
    right_rectified.shape[1] / right_image.shape[1],
)

# Preprocess images for the model
left_tensor, right_tensor = images_to_tensors(left_rectified, right_rectified)

# show rectified images
plt.subplot(1, 2, 1)
plt.imshow(left_rectified)
plt.title('Left Rectified Image')
plt.axis('off')
plt.subplot(1, 2, 2)
plt.imshow(right_rectified)
plt.title('Right Rectified Image')
plt.axis('off')
plt.show()

In [ ]:
left_image_mask = left_image.copy()
left_image_mask = cv2.cvtColor(left_image_mask, cv2.COLOR_BGR2RGB)
left_image_mask = cv2.resize(left_image_mask, (segmentation_inference.input_width, segmentation_inference.input_height))
left_image_mask = left_image_mask.astype(np.float32) / 255.0
left_image_mask = np.transpose(left_image_mask, (2, 0, 1))
left_image_tensor = np.expand_dims(left_image_mask, axis=0)
# left_image_to_display = Image.fromarray((np.transpose(left_image_tensor[0], (1, 2, 0)) * 255).astype(np.uint8))
# display(left_image_to_display)

In [ ]:
left_mask = segmentation_inference(left_image_tensor)
left_mask = np.squeeze(left_mask)
left_mask = 1 / (1 + np.exp(-left_mask))
left_mask = (left_mask > 0.5).astype(np.float32) * 255
left_mask = cv2.resize(
    left_mask,
    (left_image.shape[1], left_image.shape[0]),
    interpolation=cv2.INTER_LINEAR,
)
# left_mask_to_display = Image.fromarray((left_mask).astype(np.uint8))
# display(left_mask_to_display)

In [ ]:
kernel_size = 5
iterations = 6
kernel = np.ones((kernel_size, kernel_size), np.uint8)
left_mask = cv2.erode(left_mask, kernel, iterations=iterations)

mask_rectified = rectify_image(left_mask, stereo_map_left)
mask_rectified = cv2.resize(
    mask_rectified,
    (stereo_model.input_width, stereo_model.input_height),
    interpolation=cv2.INTER_LINEAR,
)

# show mask
plt.imshow(mask_rectified, cmap='gray')
plt.title('Left Mask Image')
plt.axis('off')
plt.show()

In [ ]:
# Stereo inference
disparity_map = stereo_model(left_tensor, right_tensor)

In [ ]:
# squeeze form 4D to 2D
disparity_map = disparity_map.squeeze()
# Visualize disparity map
disparity_map_viz = visualize_disparity(disparity_map, mask=mask_rectified)
# Postprocess disparity map
disparity_map_proc = postprocess_disparity(disparity_map, mask=mask_rectified)

# show disparity map with matplotlib
plt.imshow(disparity_map_viz)
plt.axis('off')
plt.show()

In [ ]:
# Create point cloud
pcd = create_point_cloud(
    disparity_map_proc, left_rectified, K, distance_between_cameras, scale=scale
)

# Save point cloud to PLY file
output_path = Path('../output/pcd/')
output_path.mkdir(parents=True, exist_ok=True)
o3d.io.write_point_cloud(str(output_path / (image_name + '.ply')), pcd)